In [5]:
%load_ext autoreload
%autoreload 2

# Look at this

=>  https://astra.datastax.com/org/284ba8db-af3b-491d-8955-b25f98923f7e/database/643b6819-50c0-43c9-a94c-ef3d85c64da4/connect

In [6]:
!python -c 'import cassandra; print (cassandra.__version__)'

3.25.0


In [8]:
from connection import spawn_session, simple_session_test
session = spawn_session(
    bundle_path='secure-connect-influencers.zip',
    tokens_path='influencers-token.json'
)
simple_session_test(session)

4.0.0.6816


In [162]:
type(session)

cassandra.cluster.Session

In [9]:
session.set_keyspace('scraping_etl')

создаём таблицу для хранения логов

In [167]:
query = """
CREATE TABLE IF NOT EXISTS exploring_logs
(
    log_date date, 
    relevance_entry boolean,
    create_ts timestamp, 
    video_id text, 
    related text, 
    content text,
    PRIMARY KEY (log_date, relevance_entry, video_id)
);
"""
# query = """
# DROP TABLE exploring_logs
# """
session.execute(query)

In [209]:
from tqdm.auto import tqdm
import json
from datetime import datetime
from cassandra.encoder import cql_quote
from cassandra.cluster import Session
from typing import List, Dict



def jsonl_feed_iter(fp):
    with open(fp, 'r') as f:
        for line in f:
            line = line.strip(']').strip('[').strip(',')
            yield json.loads(line)

def process_item(item, log_date, table_name):
    query = f"INSERT INTO {table_name} (log_date, relevance_entry, create_ts, video_id, related, content)"    
    query = query + " VALUES (%s, %s, %s, %s,  %s,  %s);"
    items = [
        log_date,
        'related' in item,
        str(int(datetime.utcnow().timestamp()*1000)), 
        item['video_id'], 
        str(item.get('related', [])), str(item.get('data', ''))
    ]
    return query, items

def batch_upload(session: Session, items: List[Dict], batch_size: int=3000):
    """
    Загружать батчи не убедивись что грузим только один parttion key некорректно.
    Детали тут
    https://docs.datastax.com/en/archived/cql/3.1/cql/cql_using/useBatch.html
    
    Вероятно нужно просто перейти на асинхронную запись.
    """
    def construct_batch(insert_statements):
        return 'BEGIN BATCH\n'+'\n'.join(insert_statements)+'APPLY BATCH;'
    items_batch = []
    for item in tqdm(items):
        query, item_entries = process_item(item, log_date='2021-07-14', table_name='exploring_logs')
        insert_statement = query % tuple((cql_quote(c) for c in item_entries))    
        if len(items_batch) < batch_size:
            items_batch.append(insert_statement)
        else:
            session.execute(construct_batch(items_batch))
            items_batch = []
        #     session.execute(query, tuple(item_entries))
    if len(items_batch):
        session.execute(construct_batch(items_batch))

feed_path = 'feed_2021_07_14.jsonl'
items = jsonl_feed_iter(feed_path)
batch_upload(session, items, batch_size=200)

0it [00:00, ?it/s]

Read data

In [210]:
query = 'SELECT * FROM exploring_logs WHERE log_date=\'2021-07-14\''
rows = session.execute(query)
results = []
for i, row in enumerate(tqdm(rows)):
    results.append(row)
output_df = pd.DataFrame(results)

0it [00:00, ?it/s]

In [211]:
import pandas as pd
items = jsonl_feed_iter(feed_path)
input_df = pd.DataFrame(list(items))

In [212]:
a = set(output_df['video_id'])
b = set(input_df['video_id'])

In [213]:
len(a), len(b), len(a & b)

(17865, 17865, 17865)

In [205]:
input_df[input_df['video_id']==list(b - a)[5]]

,video_id,related,data
4645,rdU6wDzG6nk,NaN,{'snippet': {'publishedAt': '2021-10-09T12:02:...


'c-CpwuM_foc'